# 国家/Slug 用户 Art Generation 任务分析

分析指定国家或 Bot Slug 用户的 Art Generation 任务参数，提取用户输入的图片和文本内容，生成 CSV 报告。

## Agent 运行模式

如果在 AI Agent 环境下运行（Claude Code、Cursor 等），提取为脚本后直接传参运行：

```bash
# 1. 提取代码到脚本
# 注意：notebook 中 .source 可能是 array 或 string，需要条件判断
jq -r '.cells[] | select(.cell_type=="code") | .source | if type=="array" then join("") else . end' \
  art-generation-country-analysis.ipynb > temp/art-generation-country-analysis.ts

# 2. 按国家查询（传入国家代码和天数）
deno run --no-check --allow-env --allow-net --allow-read --allow-write --allow-ffi \
  temp/art-generation-country-analysis.ts --country NL 7          # 荷兰，7天
  
deno run --no-check --allow-env --allow-net --allow-read --allow-write --allow-ffi \
  temp/art-generation-country-analysis.ts --country US,DE,FR 7   # 多国，7天

# 3. 按 Slug 查询（传入 bot slug 和天数）
deno run --no-check --allow-env --allow-net --allow-read --allow-write --allow-ffi \
  temp/art-generation-country-analysis.ts --slug my-bot-slug 7   # 指定 slug，7天

# 4. 同时指定国家和 Slug
deno run --no-check --allow-env --allow-net --allow-read --allow-write --allow-ffi \
  temp/art-generation-country-analysis.ts --country NL --slug my-bot-slug 7
```

**参数说明**：
- `--country`：按国家查询，后跟国家代码（逗号分隔，如 `NL` 或 `US,DE,FR`）
- `--slug`：按 Bot Slug 查询，后跟 bot slug（逗号分隔多个）
- 可以同时指定 `--country` 和 `--slug`，会分别生成各自的 CSV
- 最后一个数字参数：天数（1-30，默认 7）

**Output**: `./reports/art-generation-{country|slug}-{date}.csv`

## 数据源

| 数据源 | 说明 |
|--------|------|
| **Honeycomb** | `dev` / `myshell-art-web` - 获取各国家活跃用户 ID |
| **MySQL** | `my_shell_prod.art_task` - 查询任务参数详情 |

## Step 0: 环境配置

运行此 cell 初始化 MCP 连接和基础配置。

In [14]:
// === MCP 连接配置 ===
const MCP_HUB_URL = Deno.env.get("MCP_HUB_URL") || "http://52.12.230.109:3000/mcp";
const MCP_AUTH_TOKEN = (() => {
  let token = Deno.env.get("MCP_AUTH_TOKEN") || "";
  if (token && !token.startsWith("Bearer ")) token = `Bearer ${token}`;
  return token;
})();
const OUTPUT_DIR = "./reports";

// === 分析参数（通过 prompt 设置）===
let countryCodes: string[] = [];  // 国家代码列表
let slugs: string[] = [];         // Bot Slug 列表（可同时指定）
let days = 7;

// === 国家名称映射 ===
const COUNTRY_NAMES: Record<string, string> = {
  "NL": "荷兰", "DE": "德国", "FR": "法国", "RU": "俄罗斯",
  "US": "美国", "GB": "英国", "JP": "日本", "KR": "韩国",
  "BR": "巴西", "IN": "印度", "ID": "印尼", "TR": "土耳其",
  "CA": "加拿大", "IT": "意大利", "PL": "波兰", "SE": "瑞典",
  "ES": "西班牙", "SG": "新加坡", "PH": "菲律宾", "MX": "墨西哥",
  "BE": "比利时", "HK": "香港", "AR": "阿根廷", "NO": "挪威",
  "AU": "澳大利亚", "MY": "马来西亚", "FI": "芬兰", "GR": "希腊",
  "AT": "奥地利", "CL": "智利", "BG": "保加利亚", "CH": "瑞士",
  "BY": "白俄罗斯", "UA": "乌克兰", "CN": "中国", "RO": "罗马尼亚",
  "TN": "突尼斯", "HU": "匈牙利", "TH": "泰国", "KZ": "哈萨克斯坦",
};

// === MCP Client ===
let mcpClient: any = null;

// 验证
if (!MCP_AUTH_TOKEN) {
  console.error("❌ 请设置环境变量: export MCP_AUTH_TOKEN='your-token'");
} else {
  console.log("✅ 配置加载完成");
  console.log(`   MCP: ${MCP_HUB_URL}`);
}

✅ 配置加载完成
   MCP: http://52.12.230.109:3000/mcp


---
## Step 1: 连接 MCP Hub

MCP (Model Context Protocol) 是一个标准协议，用于连接各种数据源和工具。
我们通过 MCP Hub 来访问 Honeycomb 和 Bytebase。

**重要提示**：请先运行 Step 0 的配置单元格，然后再运行下面的连接代码。

In [15]:
// 连接 MCP Hub
import { Client } from "npm:@modelcontextprotocol/sdk@1.24.3/client/index.js";
import { StreamableHTTPClientTransport } from "npm:@modelcontextprotocol/sdk@1.24.3/client/streamableHttp.js";

console.log("📡 Connecting to MCP Hub...");

const transport = new StreamableHTTPClientTransport(new URL(MCP_HUB_URL), {
  requestInit: { headers: { Authorization: MCP_AUTH_TOKEN } },
});

mcpClient = new Client(
  { name: "art-country-analysis", version: "1.0.0" },
  { capabilities: { tools: {}, sampling: {} } }
);

await mcpClient.connect(transport);
console.log("✅ Connected to MCP Hub");

📡 Connecting to MCP Hub...
✅ Connected to MCP Hub


## 获取所有国家的用户统计

运行此单元格可以查看所有国家的用户数量，然后在下一个单元格中交互式选择要分析的国家：

In [16]:
// 配置国家、Slug 和时间 - 国家和 Slug 可以同时指定
// CLI 模式: 
//   deno run script.ts --country NL,DE 7
//   deno run script.ts --slug my-bot-slug 7
//   deno run script.ts --country NL --slug my-bot-slug 7
// Jupyter 模式: 通过 prompt() 交互输入

const args = typeof Deno !== "undefined" ? Deno.args : [];

if (args.length >= 1) {
  // CLI 模式：从命令行参数获取
  console.log("🎯 CLI 模式 - 从命令行参数获取配置\n");
  
  // 解析参数
  let i = 0;
  while (i < args.length) {
    if (args[i] === "--country" && args[i + 1]) {
      countryCodes = args[i + 1]
        .split(",")
        .map((c: string) => c.trim().toUpperCase())
        .filter((c: string) => c.length === 2);
      i += 2;
    } else if (args[i] === "--slug" && args[i + 1]) {
      slugs = args[i + 1]
        .split(",")
        .map((s: string) => s.trim())
        .filter((s: string) => s.length > 0);
      i += 2;
    } else if (/^\d+$/.test(args[i])) {
      // 数字参数作为天数
      days = Math.min(30, Math.max(1, parseInt(args[i])));
      i++;
    } else {
      // 兼容旧格式：第一个参数如果是国家代码格式
      const possibleCountries = args[i]
        .split(",")
        .map((c: string) => c.trim().toUpperCase())
        .filter((c: string) => c.length === 2);
      if (possibleCountries.length > 0) {
        countryCodes = possibleCountries;
      }
      i++;
    }
  }
  
} else {
  // Jupyter 模式：交互式输入
  console.log("🎯 配置分析参数\n");
  
  // 输入国家代码（可选）
  const countriesInput = prompt(
    `请输入国家代码（逗号分隔，留空跳过）\n示例: US,DE,NL,GB,FR`,
    ""
  );
  
  if (countriesInput && countriesInput.trim()) {
    countryCodes = countriesInput
      .split(",")
      .map((c: string) => c.trim().toUpperCase())
      .filter((c: string) => c.length === 2);
  }
  
  // 输入 Slug（可选）
  const slugsInput = prompt(
    `请输入 Bot Slug（逗号分隔多个，留空跳过）\n示例: my-bot-slug 或 bot1,bot2`,
    ""
  );
  
  if (slugsInput && slugsInput.trim()) {
    slugs = slugsInput
      .split(",")
      .map((s: string) => s.trim())
      .filter((s: string) => s.length > 0);
  }
  
  // 输入天数
  const defaultDays = "7";
  const daysInput = prompt(`查询天数（1-30）\n默认: ${defaultDays}`, defaultDays);
  days = Math.min(30, Math.max(1, parseInt(daysInput || defaultDays)));
}

// 显示配置
console.log(`✅ 配置完成:`);
if (countryCodes.length > 0) {
  const names = countryCodes.map(c => `${c}(${COUNTRY_NAMES[c] || c})`).join(", ");
  console.log(`   国家: ${countryCodes.length} 个 - ${names}`);
}
if (slugs.length > 0) {
  console.log(`   Slug: ${slugs.length} 个 - ${slugs.join(", ")}`);
}
if (countryCodes.length === 0 && slugs.length === 0) {
  console.log(`   ⚠️ 未指定国家或 Slug，请重新配置`);
}
console.log(`   时间范围: 最近 ${days} 天`);

🎯 配置分析参数

✅ 配置完成:
   国家: 2 个 - DE(德国), NL(荷兰)
   Slug: 1 个 - undress-generator
   时间范围: 最近 1 天


In [17]:
// 获取所有国家的用户统计
console.log("📊 获取所有国家的 Art Generation 用户统计...");
console.log(`   时间范围: 最近 ${days} 天\n`);

const countryResult = await mcpClient.callTool({
  name: "honeycomb-run_query",
  arguments: {
    environment_slug: "dev",
    dataset_slug: "myshell-art-web",
    query_spec: {
      calculations: [{ column: "auth.userId", op: "COUNT_DISTINCT" }],
      filters: [
        { column: "http.url", op: "=", value: "/v1/homepage/art/generate" },
        { column: "country", op: "exists" },  // 确保有国家信息
      ],
      breakdowns: ["country"],
      time_range: days * 86400,
      limit: 100,  // 获取前100个国家
      orders: [{ column: "auth.userId", op: "COUNT_DISTINCT", order: "descending" }],
    },
  },
});

// 解析结果
const countryText = (countryResult.content as any[])[0]?.text || "";
const countries = [];

for (const line of countryText.split("\n")) {
  // 匹配格式: | 5318 | US |
  const match = line.match(/\|\s*(\d+)\s*\|\s*([A-Z]{2})\s*\|/);
  if (match) {
    const count = parseInt(match[1]);
    const code = match[2];
    if (code && code !== "TOTAL" && code !== "OTHER") {
      countries.push({ code, count });
    }  
  }
}

// 显示结果
console.log("📊 各国家 Art Generation 用户数（降序）:");
console.log("═".repeat(50));
console.log("排名 | 国家 | 用户数 | 国家名称");
console.log("─".repeat(50));

const TOP_COUNTRIES = {
  "US": "美国", "DE": "德国", "NL": "荷兰", "IN": "印度",
  "RU": "俄罗斯", "FR": "法国", "GB": "英国", "CA": "加拿大",
  "IT": "意大利", "PL": "波兰", "BR": "巴西", "SE": "瑞典",
  "ES": "西班牙", "SG": "新加坡", "TR": "土耳其", "ID": "印尼",
  "JP": "日本", "KZ": "哈萨克斯坦", "PH": "菲律宾", "MX": "墨西哥",
  "BE": "比利时", "HK": "香港", "AR": "阿根廷", "NO": "挪威",
  "AU": "澳大利亚", "MY": "马来西亚", "FI": "芬兰", "GR": "希腊",
  "AT": "奥地利", "KR": "韩国", "CL": "智利", "BG": "保加利亚",
  "CH": "瑞士", "BY": "白俄罗斯", "UA": "乌克兰", "CN": "中国",
  "RO": "罗马尼亚", "TN": "突尼斯", "HU": "匈牙利", "TH": "泰国",
};

countries.slice(0, 30).forEach((c, i) => {
  const name = TOP_COUNTRIES[c.code] || c.code;
  console.log(`${(i+1).toString().padStart(4)} | ${c.code}   | ${c.count.toString().padStart(6)} | ${name}`);
});

console.log("─".repeat(50));
console.log(`总计: ${countries.length} 个国家\n`);

// 保存供后续使用
let countryList = countries;

📊 获取所有国家的 Art Generation 用户统计...
   时间范围: 最近 1 天

📊 各国家 Art Generation 用户数（降序）:
══════════════════════════════════════════════════
排名 | 国家 | 用户数 | 国家名称
──────────────────────────────────────────────────
   1 | US   |   1598 | 美国
   2 | DE   |    573 | 德国
   3 | IN   |    532 | 印度
   4 | SG   |    445 | 新加坡
   5 | NL   |    402 | 荷兰
   6 | FR   |    313 | 法国
   7 | GB   |    295 | 英国
   8 | SE   |    238 | 瑞典
   9 | PL   |    221 | 波兰
  10 | CA   |    198 | 加拿大
  11 | BR   |    192 | 巴西
  12 | HK   |    179 | 香港
  13 | IT   |    162 | 意大利
  14 | ES   |    162 | 西班牙
  15 | JP   |    143 | 日本
  16 | BG   |    120 | 保加利亚
  17 | AT   |    106 | 奥地利
  18 | GR   |     95 | 希腊
  19 | AR   |     78 | 阿根廷
  20 | HU   |     76 | 匈牙利
  21 | AU   |     67 | 澳大利亚
  22 | CL   |     63 | 智利
  23 | KR   |     57 | 韩国
  24 | IE   |     55 | IE
  25 | TR   |     53 | 土耳其
  26 | MY   |     53 | 马来西亚
  27 | NO   |     45 | 挪威
  28 | NZ   |     45 | NZ
  29 | CH   |     44 | 瑞士
  30 | RO   |     42 | 罗马尼亚
─

## 国家用户统计可视化

使用 `@observablehq/plot` 创建柱状图（Deno Jupyter 最佳选择，轻量且输出纯 SVG）：

In [18]:
// 使用 @observablehq/plot 创建国家用户统计柱状图
import * as Plot from "npm:@observablehq/plot";
import { document } from "jsr:@ry/jupyter-helper";

// 准备数据 - 取前15个国家
const chartData = countryList.slice(0, 15).map(c => ({
  country: c.code,
  name: COUNTRY_NAMES[c.code] || c.code,
  users: c.count,
}));

// 创建柱状图
Plot.plot({
  document,
  width: 800,
  height: 400,
  marginLeft: 60,
  marginBottom: 40,
  title: `Art Generation 用户分布 - Top 15 国家（最近 ${days} 天）`,
  x: { label: "国家", tickRotate: 0 },
  y: { label: "用户数", grid: true },
  color: { scheme: "tableau10" },
  marks: [
    Plot.barY(chartData, { 
      x: "country", 
      y: "users", 
      fill: "country",
      tip: true,
    }),
    Plot.ruleY([0]),
  ]
});

Art Generation 用户分布 - Top 15 国家（最近 1 天） 0 200 400 600 800 1,000 1,200 1,400 ↑ 用户数 BR CA DE ES FR GB HK IN IT JP NL PL SE SG US 国家

---
## Step 2: 批量分析

运行此单元格将根据配置进行分析（国家和 Slug 可同时指定）：

### 分析流程

**按国家查询**：对每个国家分别：
1. 从 Honeycomb 获取用户 ID（最多 1000 个）
2. 从 MySQL 查询任务参数
3. 解析并生成独立的 CSV 文件

**按 Slug 查询**：对每个 Bot Slug 分别：
1. 从 MySQL 根据 slug 查询 bot_id
2. 从 MySQL 查询该 bot 的任务参数
3. 解析并生成独立的 CSV 文件

**注意**: 每个查询可能需要 10-30 秒

In [19]:
// 批量分析 - 国家和 Slug 可同时指定，结果合并到一个 CSV
const allParsedTasks: any[] = [];  // 所有任务合并到这里
const analysisSummary: any[] = [];  // 每个国家/slug的统计

// ============ 按国家查询 ============
if (countryCodes.length > 0) {
  console.log(`🚀 开始按国家分析 ${countryCodes.length} 个国家...\n`);
  if (slugs.length > 0) {
    console.log(`   📌 同时按 slug_id 筛选: ${slugs.join(", ")}\n`);
  }

  for (const COUNTRY_CODE of countryCodes) {
    const COUNTRY_NAME = COUNTRY_NAMES[COUNTRY_CODE] || COUNTRY_CODE;
    
    console.log("═".repeat(60));
    console.log(`📍 分析国家: ${COUNTRY_CODE} (${COUNTRY_NAME})`);
    console.log("═".repeat(60));
    
    try {
      // Step 1: 从 Honeycomb 获取用户 ID
      console.log(`🔍 Querying Honeycomb for ${COUNTRY_CODE} users...`);
      
      const honeycombResult = await mcpClient.callTool({
        name: "honeycomb-run_query",
        arguments: {
          environment_slug: "dev",
          dataset_slug: "myshell-art-web",
          query_spec: {
            calculations: [{ op: "COUNT" }],
            filters: [
              { column: "name", op: "=", value: "http.post" },
              { column: "http.url", op: "=", value: "/v1/homepage/art/generate" },
              { column: "env.environment", op: "=", value: "release" },
              { column: "http.error_message", op: "does-not-exist" },
              { column: "country", op: "=", value: COUNTRY_CODE },
            ],
            breakdowns: ["auth.userId"],
            time_range: days * 86400,
            limit: 1000,
            orders: [{ op: "COUNT", order: "descending" }],
          },
        },
      });
      
      const honeycombText = (honeycombResult.content as any[])[0]?.text || "";
      const userIds = [];
      
      for (const line of honeycombText.split("\n")) {
        const match = line.match(/\|\s*\d+\s*\|\s*(\d+)\s*\|/);
        if (match && match[1]) {
          userIds.push(match[1]);
        }
      }
      
      console.log(`✅ Found ${userIds.length} unique users`);
      
      if (userIds.length === 0) {
        console.log(`⚠️ No users found for ${COUNTRY_CODE}, skipping...\n`);
        continue;
      }
      
      // Step 2: 从 MySQL 查询任务参数
      console.log(`🔍 Querying MySQL for task params...`);
      
      // 构建 slug_id 筛选条件
      let slugFilter = "";
      if (slugs.length > 0) {
        const slugList = slugs.map(s => `'${s}'`).join(",");
        slugFilter = `AND slug_id IN (${slugList})`;
        console.log(`   📌 Filtering by slug_id: ${slugs.join(", ")}`);
      }
      
      const sql = `
        SELECT
          '${COUNTRY_CODE}' as country,
          '${COUNTRY_NAME}' as country_name,
          user_id,
          task_id,
          task_params,
          created_date,
          bot_name,
          bot_id,
          slug_id,
          bot_run_type,
          status
        FROM my_shell_prod.art_task
        WHERE user_id IN (${userIds.join(",")})
          AND created_date >= DATE_SUB(NOW(), INTERVAL ${days} DAY)
          AND task_params IS NOT NULL
          AND task_params != ''
          AND task_params != '{}'
          ${slugFilter}
        ORDER BY created_date DESC
        LIMIT 10000;
      `;
      
      const sqlResult = await mcpClient.callTool({
        name: "bytebase-execute_sql",
        arguments: { sql: sql.trim() },
      });
      
      const sqlResponse = JSON.parse((sqlResult.content as any[])[0].text);
      
      if (!sqlResponse.success) {
        console.error(`❌ SQL failed for ${COUNTRY_CODE}: ${sqlResponse.error}`);
        continue;
      }
      
      const taskData = sqlResponse.data.rows;
      console.log(`✅ Found ${taskData.length} tasks`);
      
      if (taskData.length === 0) {
        console.log(`⚠️ No tasks found for ${COUNTRY_CODE}, skipping...\n`);
        continue;
      }
      
      // Step 3: 解析任务参数
      console.log(`📊 Parsing task params...`);
      
      let parseErrors = 0;
      
      for (const row of taskData) {
        try {
          const params = JSON.parse(row.task_params || "{}");
          const inputImg = params.input_img || params.inputImg || [];
          
          const images = [];
          const texts = [];
          
          if (Array.isArray(inputImg)) {
            for (const item of inputImg) {
              if (typeof item === "string") {
                if (item.startsWith("http://") || item.startsWith("https://")) {
                  images.push(item);
                } else if (item.trim()) {
                  texts.push(item.trim());
                }
              }
            }
          }
          
          allParsedTasks.push({
            country: row.country,
            country_name: row.country_name,
            user_id: row.user_id,
            task_id: row.task_id,
            created_date: row.created_date,
            bot_name: row.bot_name,
            bot_id: row.bot_id || params.bot_id || "",
            slug_id: row.slug_id || "",
            bot_run_type: row.bot_run_type || "",
            status: row.status,
            image_count: images.length,
            text_count: texts.length,
            has_text: texts.length > 0,
            has_image: images.length > 0,
            text_length: texts.join("").length,
            image_url_1: images[0] || "",
            image_url_2: images[1] || "",
            image_url_3: images[2] || "",
            image_url_4: images[3] || "",
            image_url_5: images[4] || "",
            text_prompts: texts.slice(0, 3).join(" | "),
          });
        } catch {
          parseErrors++;
        }
      }
      
      const countryTasks = allParsedTasks.filter(t => t.country === COUNTRY_CODE);
      console.log(`✅ Parsed ${countryTasks.length} tasks`);
      if (parseErrors > 0) console.log(`⚠️ ${parseErrors} parse errors`);
      
      // 统计摘要
      const stats = {
        country_code: COUNTRY_CODE,
        country_name: COUNTRY_NAME,
        total_tasks: countryTasks.length,
        tasks_with_images: countryTasks.filter((r) => r.has_image).length,
        tasks_with_text: countryTasks.filter((r) => r.has_text).length,
        unique_users: new Set(countryTasks.map((r) => r.user_id)).size,
        unique_bots: new Set(countryTasks.map((r) => r.bot_id).filter((id) => id)).size,
        unique_slugs: new Set(countryTasks.map((r) => r.slug_id).filter((id) => id)).size,
      };
      
      analysisSummary.push(stats);
      
      console.log(`\n📊 摘要统计:`);
      console.log(`   总任务数:     ${stats.total_tasks}`);
      console.log(`   独立用户数:   ${stats.unique_users}`);
      console.log(`   使用的 Bot:   ${stats.unique_bots}`);
      console.log(`   使用的 Slug:  ${stats.unique_slugs}`);
      console.log(`   含图片任务:   ${stats.tasks_with_images} (${(stats.tasks_with_images / stats.total_tasks * 100).toFixed(1)}%)`);
      console.log(`   含文本任务:   ${stats.tasks_with_text} (${(stats.tasks_with_text / stats.total_tasks * 100).toFixed(1)}%)`);
      console.log("");
      
    } catch (error) {
      console.error(`❌ Error processing ${COUNTRY_CODE}: ${error.message}\n`);
    }
  }
}

// ============ 单独按 Slug 查询（仅当没有指定国家时）============
if (slugs.length > 0 && countryCodes.length === 0) {
  console.log(`🚀 开始按 Slug 分析 ${slugs.length} 个 Bot...\n`);
  
  for (const slug of slugs) {
    console.log("═".repeat(60));
    console.log(`📍 分析 Slug: ${slug}`);
    console.log("═".repeat(60));
    
    try {
      console.log(`🔍 Querying MySQL for tasks with slug_id: ${slug}...`);
      
      const sql = `
        SELECT
          '' as country,
          '' as country_name,
          slug_id,
          bot_name,
          user_id,
          task_id,
          task_params,
          created_date,
          bot_id,
          bot_run_type,
          status
        FROM my_shell_prod.art_task
        WHERE slug_id = '${slug}'
          AND created_date >= DATE_SUB(NOW(), INTERVAL ${days} DAY)
          AND task_params IS NOT NULL
          AND task_params != ''
          AND task_params != '{}'
        ORDER BY created_date DESC
        LIMIT 10000;
      `;
      
      const sqlResult = await mcpClient.callTool({
        name: "bytebase-execute_sql",
        arguments: { sql: sql.trim() },
      });
      
      const sqlResponse = JSON.parse((sqlResult.content as any[])[0].text);
      
      if (!sqlResponse.success) {
        console.error(`❌ SQL failed for ${slug}: ${sqlResponse.error}`);
        continue;
      }
      
      const taskData = sqlResponse.data.rows;
      console.log(`✅ Found ${taskData.length} tasks`);
      
      if (taskData.length === 0) {
        console.log(`⚠️ No tasks found for slug_id: ${slug}, skipping...\n`);
        continue;
      }
      
      const botName = taskData[0]?.bot_name || slug;
      console.log(`   Bot 名称: ${botName}`);
      console.log(`📊 Parsing task params...`);
      
      let parseErrors = 0;
      
      for (const row of taskData) {
        try {
          const params = JSON.parse(row.task_params || "{}");
          const inputImg = params.input_img || params.inputImg || [];
          
          const images = [];
          const texts = [];
          
          if (Array.isArray(inputImg)) {
            for (const item of inputImg) {
              if (typeof item === "string") {
                if (item.startsWith("http://") || item.startsWith("https://")) {
                  images.push(item);
                } else if (item.trim()) {
                  texts.push(item.trim());
                }
              }
            }
          }
          
          allParsedTasks.push({
            country: row.country || "",
            country_name: row.country_name || "",
            slug_id: row.slug_id,
            bot_name: row.bot_name,
            user_id: row.user_id,
            task_id: row.task_id,
            created_date: row.created_date,
            bot_id: row.bot_id || params.bot_id || "",
            bot_run_type: row.bot_run_type || "",
            status: row.status,
            image_count: images.length,
            text_count: texts.length,
            has_text: texts.length > 0,
            has_image: images.length > 0,
            text_length: texts.join("").length,
            image_url_1: images[0] || "",
            image_url_2: images[1] || "",
            image_url_3: images[2] || "",
            image_url_4: images[3] || "",
            image_url_5: images[4] || "",
            text_prompts: texts.slice(0, 3).join(" | "),
          });
        } catch {
          parseErrors++;
        }
      }
      
      const slugTasks = allParsedTasks.filter(t => t.slug_id === slug);
      console.log(`✅ Parsed ${slugTasks.length} tasks`);
      if (parseErrors > 0) console.log(`⚠️ ${parseErrors} parse errors`);
      
      const stats = {
        slug: slug,
        bot_name: botName,
        total_tasks: slugTasks.length,
        tasks_with_images: slugTasks.filter((r) => r.has_image).length,
        tasks_with_text: slugTasks.filter((r) => r.has_text).length,
        unique_users: new Set(slugTasks.map((r) => r.user_id)).size,
      };
      
      analysisSummary.push(stats);
      
      console.log(`\n📊 摘要统计:`);
      console.log(`   Bot 名称:     ${stats.bot_name}`);
      console.log(`   总任务数:     ${stats.total_tasks}`);
      console.log(`   独立用户数:   ${stats.unique_users}`);
      console.log(`   含图片任务:   ${stats.tasks_with_images} (${(stats.tasks_with_images / stats.total_tasks * 100).toFixed(1)}%)`);
      console.log(`   含文本任务:   ${stats.tasks_with_text} (${(stats.tasks_with_text / stats.total_tasks * 100).toFixed(1)}%)`);
      console.log("");
      
    } catch (error) {
      console.error(`❌ Error processing ${slug}: ${error.message}\n`);
    }
  }
}

// ============ 生成合并的 CSV 文件 ============
if (allParsedTasks.length > 0) {
  console.log("═".repeat(60));
  console.log("💾 生成合并的 CSV 文件...");
  console.log("═".repeat(60));
  
  const headers = [
    "country", "country_name", "user_id", "task_id", "created_date",
    "bot_name", "bot_id", "slug_id", "bot_run_type", "status",
    "image_count", "text_count", "has_text", "has_image", "text_length",
    "image_url_1", "image_url_2", "image_url_3", "image_url_4", "image_url_5",
    "text_prompts",
  ];
  
  const csvRows = allParsedTasks.map((task) => {
    return headers.map((h) => {
      let v = (task as any)[h];
      if (v === null || v === undefined) v = "";
      else if (typeof v === "boolean") v = v ? "TRUE" : "FALSE";
      else v = String(v);
      if (v.includes(",") || v.includes('"') || v.includes("\n")) {
        v = `"${v.replace(/"/g, '""')}"`;
      }
      return v;
    }).join(",");
  });
  
  const csv = [headers.join(","), ...csvRows].join("\n");
  
  try { await Deno.mkdir(OUTPUT_DIR, { recursive: true }); } catch {}
  
  const today = new Date().toISOString().split("T")[0];
  
  // 生成文件名
  let fileNameParts = ["art-generation"];
  if (countryCodes.length > 0) {
    fileNameParts.push(countryCodes.map(c => c.toLowerCase()).join("-"));
  }
  if (slugs.length > 0) {
    fileNameParts.push(slugs.map(s => s.replace(/[^a-zA-Z0-9-_]/g, "_")).join("-"));
  }
  fileNameParts.push(today);
  
  const outputFile = `${OUTPUT_DIR}/${fileNameParts.join("-")}.csv`;
  await Deno.writeTextFile(outputFile, csv);
  
  console.log(`\n✅ CSV saved: ${outputFile}`);
  console.log(`   总任务数: ${allParsedTasks.length}`);
  console.log(`   文件大小: ${(csv.length / 1024).toFixed(1)} KB`);
}

// ============ 总体摘要 ============
console.log("\n" + "═".repeat(60));
console.log("📊 批量分析完成 - 总体摘要");
console.log("═".repeat(60));

if (analysisSummary.length > 0 && countryCodes.length > 0) {
  console.log("\n📍 各国家统计:");
  if (slugs.length > 0) {
    console.log(`   (已按 slug_id 筛选: ${slugs.join(", ")})`);
  }
  console.log("─".repeat(60));
  console.log("国家 | 任务数 | 用户数 | Bot数 | Slug数 | 图片% | 文本%");
  console.log("─".repeat(60));

  for (const result of analysisSummary.filter(r => r.country_code)) {
    const imgPct = (result.tasks_with_images / result.total_tasks * 100).toFixed(0);
    const txtPct = (result.tasks_with_text / result.total_tasks * 100).toFixed(0);
    console.log(
      `${result.country_code}   | ${result.total_tasks.toString().padStart(6)} | ` +
      `${result.unique_users.toString().padStart(6)} | ${result.unique_bots.toString().padStart(5)} | ` +
      `${(result.unique_slugs || 0).toString().padStart(6)} | ` +
      `${imgPct.padStart(4)}% | ${txtPct.padStart(4)}%`
    );
  }
}

// 总计
console.log("─".repeat(60));
const totalStats = {
  total_tasks: allParsedTasks.length,
  tasks_with_images: allParsedTasks.filter((r) => r.has_image).length,
  tasks_with_text: allParsedTasks.filter((r) => r.has_text).length,
  unique_users: new Set(allParsedTasks.map((r) => r.user_id)).size,
  unique_countries: new Set(allParsedTasks.map((r) => r.country).filter(c => c)).size,
  unique_slugs: new Set(allParsedTasks.map((r) => r.slug_id).filter(s => s)).size,
};

console.log(`\n📊 总计:`);
console.log(`   总任务数:     ${totalStats.total_tasks}`);
console.log(`   独立用户数:   ${totalStats.unique_users}`);
console.log(`   涉及国家:     ${totalStats.unique_countries}`);
console.log(`   涉及 Slug:    ${totalStats.unique_slugs}`);
console.log(`   含图片任务:   ${totalStats.tasks_with_images} (${(totalStats.tasks_with_images / totalStats.total_tasks * 100).toFixed(1)}%)`);
console.log(`   含文本任务:   ${totalStats.tasks_with_text} (${(totalStats.tasks_with_text / totalStats.total_tasks * 100).toFixed(1)}%)`);

console.log(`\n✅ 已生成 1 个合并的 CSV 文件在 ${OUTPUT_DIR} 目录`);

// 保存结果供后续使用
const allCountryResults = allParsedTasks;

// CLI 模式下退出进程
if (Deno.args.length > 0) Deno.exit(0);

🚀 开始按国家分析 2 个国家...

   📌 同时按 slug_id 筛选: undress-generator

════════════════════════════════════════════════════════════
📍 分析国家: DE (德国)
════════════════════════════════════════════════════════════
🔍 Querying Honeycomb for DE users...
✅ Found 99 unique users
🔍 Querying MySQL for task params...
   📌 Filtering by slug_id: undress-generator
✅ Found 140 tasks
📊 Parsing task params...
✅ Parsed 140 tasks

📊 摘要统计:
   总任务数:     140
   独立用户数:   23
   使用的 Bot:   1
   使用的 Slug:  1
   含图片任务:   140 (100.0%)
   含文本任务:   140 (100.0%)

════════════════════════════════════════════════════════════
📍 分析国家: NL (荷兰)
════════════════════════════════════════════════════════════
🔍 Querying Honeycomb for NL users...
✅ Found 99 unique users
🔍 Querying MySQL for task params...
   📌 Filtering by slug_id: undress-generator
✅ Found 69 tasks
📊 Parsing task params...
✅ Parsed 69 tasks

📊 摘要统计:
   总任务数:     69
   独立用户数:   18
   使用的 Bot:   1
   使用的 Slug:  1
   含图片任务:   69 (100.0%)
   含文本任务:   69 (100.0%)

═════════════

---
## 完成！

CSV 文件已保存到 `reports/` 目录，每个国家或 Slug 一个文件。

### 使用说明

1. **配置参数**：在 Step 0 中设置 MCP 连接参数
2. **连接 MCP**：运行 Step 1 连接到 MCP Hub
3. **获取国家列表**：（可选）运行 "获取所有国家的用户统计" 查看所有可用国家
4. **配置查询**：运行配置单元格，分别输入国家代码和/或 Slug（两者可同时指定）
5. **批量分析**：运行 Step 2 对所有配置的国家和 Slug 进行分析

### 查询维度

- **国家**：通过 Honeycomb 获取指定国家的用户，再查询其任务
- **Slug**：直接根据 Bot Slug 从 MySQL 查询所有使用该 Bot 的任务
- 两者可以同时指定，会分别生成各自的 CSV 文件

### 输出文件

- **国家**：`art-generation-{country_code}-{date}.csv`
- **Slug**：`art-generation-slug-{slug}-{date}.csv`
- 位置：`./reports/` 目录
- 内容：包含用户ID、任务ID、图片URL、文本提示等详细信息